### Anomoly Detection: Group Project

    Created By: Mijail Mariano

    23AUGUST2022

In [32]:
# notebook dependencies
%matplotlib inline
import matplotlib as mlp
mlp.rcParams['figure.dpi'] = 300

import pandas as pd
import numpy as np
import os

# visualization libraries/modules
import matplotlib.pyplot as plt
import seaborn as sns

# pycaret import
from pycaret.anomaly import *

# plotly import
# import plotly.express as px
# import plotly.io as pio
# pio.renderers.default = "notebook_connected"

# created module
import mm_prepare
from mm_prepare import get_logs_dataset

# skimpy module to clean column names
from skimpy import clean_columns

# regular expression module
import re

import env_mm
from env_mm import user, password, host, get_connection

In [33]:
# importing the data

df = get_logs_dataset()
df.head()

df shape: (900223, 11)


,date,time,endpoint,user_id,cohort_id,ip,name,slack,start_date,end_date,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2.0


In [34]:
# dataframe information

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900223 entries, 0 to 900222
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        900223 non-null  object 
 1   time        900223 non-null  object 
 2   endpoint    900222 non-null  object 
 3   user_id     900223 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          900223 non-null  object 
 6   name        847330 non-null  object 
 7   slack       847330 non-null  object 
 8   start_date  847330 non-null  object 
 9   end_date    847330 non-null  object 
 10  program_id  847330 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 75.5+ MB


In [35]:
'''function that returns the endpoint class and topic'''
def get_endpoint_targets(df):

    topics = df["endpoint"].str.split("/", n = 1, expand = True). \
                rename(columns = {0: "class", 1: "topic"})
    
    new_df = pd.concat([df, topics], axis = 1)

    # returns the new df w/endpoint class and topics
    return new_df

In [36]:
# testing out the function/syntax

df["endpoint"].str.split("/", n = 1, expand = True).rename(columns = {0: "class", 1: "topic"})

,class,topic
0,,
1,java-ii,None
2,java-ii,object-oriented-programming
3,slides,object_oriented_programming
4,javascript-i,conditionals
...,...,...
900218,jquery,personal-site
900219,jquery,mapbox-api
900220,jquery,ajax/weather-map
900221,anomaly-detection,discrete-probabilistic-methods


In [37]:
# deploying the function 

df = mm_prepare.get_endpoint_targets(df)
df.head()

df shape: (900223, 13)


,date,time,endpoint,user_id,cohort_id,ip,name,slack,start_date,end_date,program_id,class,topic
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0,,
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0,java-ii,None
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0,java-ii,object-oriented-programming
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0,slides,object_oriented_programming
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2.0,javascript-i,conditionals


In [38]:
# cleaning "class" labels 

df = mm_prepare.clean_lesson(df)
df[df["class"].str.contains("fundamentals") == True]["class"].unique() # checks out!

array(['fundamentals'], dtype=object)

In [39]:
# expressed another way
# str.split() method to expand endpoints on "/"

df_test = pd.DataFrame()

df_test[["class", "topic"]] = df["endpoint"].str.split("/", n = 1, expand = True)
df_test.head()

,class,topic
0,,
1,java-ii,None
2,java-ii,object-oriented-programming
3,slides,object_oriented_programming
4,javascript-i,conditionals


In [40]:
# using the clean date function

df = mm_prepare.clean_dates(df)
df.head()

new df shape: (900223, 12)


,endpoint,user_id,ip,name,slack,start_date,end_date,program_id,class,topic,day,month
2018-01-26 09:55:03,/,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0,,,Friday,January
2018-01-26 09:56:02,java-ii,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0,java-ii,None,Friday,January
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0,java-ii,object-oriented-programming,Friday,January
2018-01-26 09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,1.0,slides,object_oriented_programming,Friday,January
2018-01-26 09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,2.0,javascript-i,conditionals,Friday,January


In [41]:
# initial data familiarization

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 09:55:03 to 2021-04-21 16:44:39
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   endpoint    900222 non-null  object 
 1   user_id     900223 non-null  int64  
 2   ip          900223 non-null  object 
 3   name        847330 non-null  object 
 4   slack       847330 non-null  object 
 5   start_date  847330 non-null  object 
 6   end_date    847330 non-null  object 
 7   program_id  847330 non-null  float64
 8   class       900222 non-null  object 
 9   topic       731934 non-null  object 
 10  day         900223 non-null  object 
 11  month       900223 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 89.3+ MB


In [42]:
# setting the program_id to object type

df[["user_id", "program_id"]] = df[["user_id", "program_id"]].astype(object)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 09:55:03 to 2021-04-21 16:44:39
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   endpoint    900222 non-null  object
 1   user_id     900223 non-null  object
 2   ip          900223 non-null  object
 3   name        847330 non-null  object
 4   slack       847330 non-null  object
 5   start_date  847330 non-null  object
 6   end_date    847330 non-null  object
 7   program_id  847330 non-null  object
 8   class       900222 non-null  object
 9   topic       731934 non-null  object
 10  day         900223 non-null  object
 11  month       900223 non-null  object
dtypes: object(12)
memory usage: 89.3+ MB


In [43]:
# mapping the codeup program type by program_id

df = mm_prepare.map_program_id(df)
df.head()

,endpoint,user_id,ip,name,slack,start_date,end_date,class,topic,day,month,program_type
2018-01-26 09:55:03,/,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,,,Friday,January,FS_PHP_program
2018-01-26 09:56:02,java-ii,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,java-ii,None,Friday,January,FS_PHP_program
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,java-ii,object-oriented-programming,Friday,January,FS_PHP_program
2018-01-26 09:56:06,slides/object_oriented_programming,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,slides,object_oriented_programming,Friday,January,FS_PHP_program
2018-01-26 09:56:24,javascript-i/conditionals,2,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,javascript-i,conditionals,Friday,January,FS_JAVA_program


In [44]:
# checking the number of missing values per column

df.isnull().sum().sort_values(ascending = False)

topic           168289
name             52893
slack            52893
start_date       52893
end_date         52893
program_type     52893
endpoint             1
class                1
user_id              0
ip                   0
day                  0
month                0
dtype: int64

In [45]:
# checking the percentage of nulls per column 

df.isnull().mean().sort_values(ascending = False).round(2)

topic           0.19
name            0.06
slack           0.06
start_date      0.06
end_date        0.06
program_type    0.06
endpoint        0.00
class           0.00
user_id         0.00
ip              0.00
day             0.00
month           0.00
dtype: float64

In [69]:
# are the features/columns for slack, cohort name, start date, end date, program consistent across rows?

df.loc[df[["name", "slack", "start_date", "end_date", "program_type"]].isnull().apply(lambda x: all(x), axis=1)]

,endpoint,user_id,ip,name,slack,start_date,end_date,class,topic,day,month,program_type
2018-01-26 16:46:16,/,48,97.105.19.61,NaN,NaN,NaN,NaN,NaN,NaN,Friday,January,NaN
2018-01-26 16:46:24,spring/extra-features/form-validation,48,97.105.19.61,NaN,NaN,NaN,NaN,spring,extra-features/form-validation,Friday,January,NaN
2018-01-26 17:54:24,/,48,97.105.19.61,NaN,NaN,NaN,NaN,NaN,NaN,Friday,January,NaN
2018-01-26 18:32:03,/,48,97.105.19.61,NaN,NaN,NaN,NaN,NaN,NaN,Friday,January,NaN
2018-01-26 18:32:17,mysql/relationships/joins,48,97.105.19.61,NaN,NaN,NaN,NaN,mysql,relationships/joins,Friday,January,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-21 12:49:00,javascript-ii,717,136.50.102.126,NaN,NaN,NaN,NaN,javascript-ii,None,Wednesday,April,NaN
2021-04-21 12:49:02,javascript-ii/es6,717,136.50.102.126,NaN,NaN,NaN,NaN,javascript-ii,es6,Wednesday,April,NaN
2021-04-21 12:51:27,javascript-ii/map-filter-reduce,717,136.50.102.126,NaN,NaN,NaN,NaN,javascript-ii,map-filter-reduce,Wednesday,April,NaN
2021-04-21 12:52:37,javascript-ii/promises,717,136.50.102.126,NaN,NaN,NaN,NaN,javascript-ii,promises,Wednesday,April,NaN


In [73]:
# let's create a new dataframe from these records

fifty_3 = df.loc[df[["name", "slack", "start_date", "end_date", "program_type"]].isnull().apply(lambda x: all(x), axis=1)]

In [87]:
# let's print the frequency of unique values

container = []

for col in fifty_3.columns:
    # where "1.0" represents total null percentage in column
    if fifty_3[col].isnull().mean() != 1.0:

        metric = {  
            "feature": col,
            "data_type": df[col].dtype,
            "unique_values": df[col].nunique(),
            "1st_freq_observation": df[col].value_counts().idxmax(),
            "total_observations": df[col].value_counts().max(),
            "2nd_most_frequent": df[col].value_counts().index[1],
            "total_observations": df[col].value_counts().values[1],
        }

        container.append(metric)

pd.DataFrame(container).sort_values("total_observations", ascending = False).reset_index(drop = True)

,feature,data_type,unique_values,1st_freq_observation,total_observations,2nd_most_frequent
0,day,object,7,Monday,179515,Tuesday
1,month,object,12,March,97538,April
2,class,object,734,javascript-i,84935,html-css
3,ip,object,5531,97.105.19.58,61662,97.105.19.61
4,endpoint,object,2313,/,19519,search/search_index.json
5,topic,object,1284,search_index.json,17323,introduction
6,user_id,int64,981,11,16347,64


In [71]:
# fifty-3 value_counts() -- where returned values indicate same nulls across identified features
df[["name", "slack", "start_date", "end_date", "program_type"]].isnull().apply(lambda x: all(x), axis=1).value_counts()

False    847329
True      52893
dtype: int64

In [47]:
# let's view the only (1) missing values in endpoint and class

df[df["endpoint"].isnull()]

,endpoint,user_id,ip,name,slack,start_date,end_date,class,topic,day,month,program_type
2020-04-08 09:25:18,NaN,586,72.177.240.51,Curie,#curie,2020-02-03,2020-07-07,NaN,NaN,Wednesday,April,DS_program


In [48]:
# let's view the only (1) missing values in endpoint and class

df[df["class"].isnull()]

,endpoint,user_id,ip,name,slack,start_date,end_date,class,topic,day,month,program_type
2020-04-08 09:25:18,NaN,586,72.177.240.51,Curie,#curie,2020-02-03,2020-07-07,NaN,NaN,Wednesday,April,DS_program


In [49]:
# let's examing user_id 586

df_586 = df[df["user_id"] == 586]
df_586.shape

(983, 12)

In [50]:
# how many ip addresses has this user_id used?

df_586["ip"].nunique()

40

In [51]:
# what unique endpoint-classes have they looked at?

pd.Series(df_586["class"].unique().tolist())

0                      
1              appendix
2            regression
3          fundamentals
4          storytelling
5                   sql
6                search
7                python
8                 stats
9                   NaN
10       classification
11    anomaly-detection
12      advanced-topics
13           clustering
14       distributed-ml
15           timeseries
16                  nlp
dtype: object

In [52]:
# cleaning columns with empty class and topic observations
# ~50k nulls in "class" column

df = df.replace(r'^\s*$', np.nan, regex=True)

len(df[df["class"].isnull()])

50314

In [53]:
df.shape

(900223, 12)

In [54]:
# drop single missing values in endpoint and class (same record)

df = df.dropna(subset = "endpoint")
df.isnull().sum()

endpoint             0
user_id              0
ip                   0
name             52893
slack            52893
start_date       52893
end_date         52893
class            50313
topic           218601
day                  0
month                0
program_type     52893
dtype: int64

In [55]:
# what percentage of missing values makeup ea. column/feature?

df.isnull().mean().sort_values(ascending = False).round(3)

topic           0.243
name            0.059
slack           0.059
start_date      0.059
end_date        0.059
program_type    0.059
class           0.056
endpoint        0.000
user_id         0.000
ip              0.000
day             0.000
month           0.000
dtype: float64

----

### ``Analyzing Missing Values``

In [56]:
# let's examine missing values 
# ~99K records with >= 1 Feature Null value

col_lst = [col for col in df.columns if "topic" not in col] # this column has an explained high-level of Nulls

null_df = df[col_lst].loc[df[col_lst].isnull().any(axis = 1)]

print(f'null dataframe shape: {null_df.shape}')
null_df.head()

null dataframe shape: (98747, 11)


,endpoint,user_id,ip,name,slack,start_date,end_date,class,day,month,program_type
2018-01-26 09:55:03,/,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,NaN,Friday,January,FS_PHP_program
2018-01-26 10:00:37,/,6,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,NaN,Friday,January,FS_JAVA_program
2018-01-26 10:14:47,/,11,97.105.19.61,Arches,#arches,2014-02-04,2014-04-22,NaN,Friday,January,FS_PHP_program
2018-01-26 10:14:53,/,6,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,NaN,Friday,January,FS_JAVA_program
2018-01-26 10:18:22,/,10,97.105.19.61,Sequoia,#sequoia,2017-09-27,2018-02-15,NaN,Friday,January,FS_JAVA_program


In [57]:
# let's examine unique values in ea. feature/vairable

mm_prepare.print_variable_info(null_df)

feature: endpoint
feature type: object
number of unique values: 1112
unique values: ['/' 'spring/extra-features/form-validation' 'mysql/relationships/joins'
 ... 'classification/classical_programming_vs_machine_learning.jpeg'
 'distributed-ml/explore' 'appendix/further-reading/jquery/effects']
value counts: /                                                                                           50313
search/search_index.json                                                                     1985
javascript-i                                                                                  780
toc                                                                                           706
spring                                                                                        641
java-iii                                                                                      567
html-css                                                                                      508
java-

In [58]:
null_df.head()

,endpoint,user_id,ip,name,slack,start_date,end_date,class,day,month,program_type
2018-01-26 09:55:03,/,1,97.105.19.61,Hampton,#hampton,2015-09-22,2016-02-06,NaN,Friday,January,FS_PHP_program
2018-01-26 10:00:37,/,6,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,NaN,Friday,January,FS_JAVA_program
2018-01-26 10:14:47,/,11,97.105.19.61,Arches,#arches,2014-02-04,2014-04-22,NaN,Friday,January,FS_PHP_program
2018-01-26 10:14:53,/,6,97.105.19.61,Teddy,#teddy,2018-01-08,2018-05-17,NaN,Friday,January,FS_JAVA_program
2018-01-26 10:18:22,/,10,97.105.19.61,Sequoia,#sequoia,2017-09-27,2018-02-15,NaN,Friday,January,FS_JAVA_program


In [59]:
null_df["slack"].value_counts()

#              8307
#darden        2980
#voyageurs     2101
#fortuna       2038
#teddy         1828
#zion          1798
#ganymede      1727
#curie         1712
#jupiter       1696
#ceres         1653
#ulysses       1641
#deimos        1467
#hyperion      1365
#apex          1346
#europa        1295
#andromeda     1174
#wrangell      1132
#easley        1115
#yosemite       981
#betelgeuse     955
#kalypso        939
#xanadu         924
#bash           772
#marco          699
#sequoia        630
#arches         626
#florence       584
#luna           491
#neptune        472
#olympic        249
#kings          219
#hampton        210
#quincy         151
#pinnacles      149
#oberon         131
#lassen         125
#glacier         51
#niagara         37
#mammoth         33
#ike             19
#badlands        17
#joshua           8
#franklin         4
#everglades       1
#apollo           1
#denali           1
Name: slack, dtype: int64

In [60]:
# creating a dataframe to capture the highest feature frequency

container = []

for col in null_df.columns:
    
    if col == "endpoint" or col == "slack":
        
        metric = {  
            "feature": col,
            "data_type": null_df[col].dtype,
            "unique_values": null_df[col].nunique(),
            "most_freq_observation": null_df[col].value_counts().index[1],
            "total_observations": null_df[col].value_counts()[1].max()
        }

        container.append(metric)

    elif col != "class":
        
        metric = {  
            "feature": col,
            "data_type": df[col].dtype,
            "unique_values": df[col].nunique(),
            "most_freq_observation": df[col].value_counts().idxmax(),
            "total_observations": df[col].value_counts().max()
        }

        container.append(metric)

    else:

        metric = {  
            "feature": col,
            "data_type": df[col].dtype,
            "unique_values": np.nan,
            "most_freq_observation": np.nan,
            "total_observations": np.nan
        }

        container.append(metric)


freq_df = pd.DataFrame(container).sort_values("total_observations", ascending = False).reset_index(drop = True)
freq_df

,feature,data_type,unique_values,most_freq_observation,total_observations
0,program_type,object,4.0,FS_JAVA_program,713365.0
1,ip,object,5531.0,97.105.19.58,284579.0
2,day,object,7.0,Monday,185188.0
3,month,object,12.0,March,110361.0
4,start_date,object,44.0,2014-02-04,92921.0
5,name,object,47.0,Staff,84031.0
6,end_date,object,45.0,2014-02-04,84031.0
7,user_id,int64,981.0,11,17913.0
8,slack,object,46.0,#darden,2980.0
9,endpoint,object,1112.0,search/search_index.json,1985.0


In [61]:
# Who is using ip addres "97.105.19.58" ?

null_df[null_df["ip"] == "97.105.19.58"]["user_id"].unique()

array([183, 205, 195, 188, 196, 145, 194, 193, 203, 190, 185, 198, 136,
       186, 207, 184, 215, 204, 189, 127, 155, 200, 208, 124, 119, 128,
       199,  64, 118,  11, 187, 134, 157, 201, 206, 182, 130,   1, 197,
       131, 135,  41,  53, 192,  26, 211, 219, 220, 221, 222, 223, 224,
       225, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238,
       239, 218, 240, 191, 146,  28,  40, 226, 248, 253, 249, 256, 258,
       259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271,
       272, 273, 274, 275, 276, 277, 257, 250,  58,  88, 278, 251, 282,
       287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299,
       300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 312, 314,
       315, 164, 321, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332,
       333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345,
       346, 347, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359,
       360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 371, 37

In [62]:
# who is the most frequent user of this ip address?

null_df[null_df["ip"] == "97.105.19.58"]["user_id"].value_counts()

354    2065
363    1876
368    1261
355    1208
349    1162
       ... 
519       1
250       1
28        1
26        1
629       1
Name: user_id, Length: 399, dtype: int64

----
#### 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?